In [0]:
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from random import randrange, uniform
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
Using TensorFlow backend.


In [0]:
pd.

In [0]:
mrkt = pd.read_csv("https://raw.githubusercontent.com/ankitactive/Email_marketing_conversion/master/marketing_tr.csv")

# **Get Dummies**

In [0]:
mrkt.isna().sum()
mrkt['schooling'] = mrkt['schooling'].replace(np.nan,"asd")
mrkt['custAge'] = mrkt['custAge'].replace(np.nan,mrkt['custAge'].mean())
mrkt['day_of_week'] = mrkt['day_of_week'].replace(np.nan,mrkt['custAge'].mean())


In [0]:
mrkt = pd.get_dummies(mrkt)
#mrkt['responded_yes'] = mrkt['responded_yes'].replace(0, 'No')
#mrkt['responded_yes'] = mrkt['responded_yes'].replace(1, 'Yes')


In [0]:
mrkt.shape
mrkt = mrkt.drop('responded_no',axis=1)

In [0]:
x = mrkt.values[:,0:66]
y = mrkt.values[:,66]

In [0]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .2)

# **C50 Desicion Tree**

In [0]:
c50_Model = tree.DecisionTreeClassifier(criterion='entropy').fit(x_train,y_train)

In [0]:
C50_Predictions = c50_Model.predict(x_test)
C50_Predictions

array(['No', 'Yes', 'Yes', ..., 'Yes', 'No', 'No'], dtype=object)

In [0]:
cm = confusion_matrix(y_test,C50_Predictions)

In [0]:
CM = pd.crosstab(y_test,C50_Predictions)

In [0]:
CM

col_0,No,Yes
row_0,,
No,1205,117
Yes,108,53


In [0]:
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

In [0]:
accuracy_score(y_test,C50_Predictions) #FNR 67.08

0.8482805124747134

# **Random Forest**

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
?RandomForestClassifier

In [0]:
RF = RandomForestClassifier(n_estimators = 80).fit(x_train,y_train)

In [0]:
RFprediction = RF.predict(x_test)

In [0]:
RFprediction

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [0]:
cm = confusion_matrix(y_test,RFprediction)

In [0]:
CM = pd.crosstab(y_test,RFprediction)

In [0]:
CM

col_0,No,Yes
row_0,,
No,1298,31
Yes,112,42


In [0]:
accuracy_score(y_test,RFprediction) #FNR  71.41

0.9035738368172623

In [0]:
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

In [0]:
FNR = FN/(FN+TP)
FNR

0.7272727272727273

# **Logistic Regression**

In [0]:
mrkt['responded'] = mrkt['responded'].replace('no', 0)
mrkt['responded'] = mrkt['responded'].replace('yes', 1)
mrkt = mrkt.dropna()

In [0]:
logit = pd.DataFrame(mrkt["responded"])

In [0]:
cnames =  ["custAge", "campaign", "pdays", "previous", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m","nr.employed", "pmonths", "pastEmail"]

In [0]:
logit = logit.join(mrkt[cnames])

In [0]:
cat_names = ["profession", "marital", "schooling", "default","contact", "month", "poutcome"]

for i in cat_names:
  temp = pd.get_dummies(mrkt[i], prefix = i)
  logit = logit.join(temp)

In [0]:
Sample_Index = np.random.rand(len(logit)) < 0.8
train = logit[Sample_Index]
test = logit[~Sample_Index]

In [0]:
import statsmodels.api as sm

In [0]:
tt = train.columns[1:31]

In [0]:
logitLR = sm.Logit(train['responded'], train[tt]).fit()

Optimization terminated successfully.
         Current function value: 0.264144
         Iterations 8


In [0]:
logitLR.summary()

In [0]:
predict = logitLR.predict(test[tt])

In [0]:

for i in range (0,predict.count()-1):
  if predict.iloc[i] > 0.5:
    predict.iloc[i] = 1
  else:
    predict.iloc[i] = 0


In [0]:
CM = pd.crosstab(test['responded'],predict)

In [0]:
CM

col_0,0.0,0.034472333227042254,1.0
responded,,,
0,581,0,12
1,58,1,20
